In [7]:

from tqdm import tqdm
import random
random.seed(1337)
import matplotlib.pyplot as plt
import argparse
import numpy as np
np.random.seed(1337)
import pandas as pd
import torch
from framepool import *
from util import *
import keras

import random
random.seed(1337)
import os

import scipy.stats as stats

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

tf.compat.v1.enable_eager_execution()

import pandas as pd
import numpy as np
import requests, sys

DATA = './../../data/utrdb2.csv'
BATCH_SIZE = 64
LR = 0.001
TASK = "te"
GPU = '-1'
STEPS = 100

if GPU == '-1':
    device = 'cpu'
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = GPU
    device = 'cuda'
    if ',' in GPU:
        device = 'cuda:1'

def prepare_mttrans(seqs):
    seqs_init = torch.tensor(np.array(one_hot_all_motif(seqs),dtype=np.float32))

    seqs_init = torch.transpose(seqs_init, 1, 2)
    seqs_init = torch.tensor(seqs_init,dtype=torch.float32).to(device)
    return seqs_init

def prepare_framepool(seqs):
    return tf.convert_to_tensor(np.array([encode_seq_framepool(seq) for seq in seqs]),dtype=tf.float32)



DIM = 40
SEQ_LEN = 128
UTR_LEN = 128
gpath = './../../models/checkpoint_3000.h5'
mrl_path = './../../models/utr_model_combined_residual_new.h5'

path = './script/checkpoint/RL_hard_share_MTL/3R/schedule_MTL-model_best_cv1.pth'
val_path = './script/checkpoint/RL_hard_share_MTL/3M/schedule_lr-model_best_cv1.pth'



if TASK == 'te':
    path = './script/checkpoint/RL_hard_share_MTL/3R/schedule_MTL-model_best_cv1.pth'
    OPT = 'TE'
else:
    path = './script/checkpoint/RL_hard_share_MTL/3M/schedule_lr-model_best_cv1.pth'
    OPT = 'FMRL'


out_folder = './outputs/'
os.makedirs(out_folder, exist_ok=True)

In [9]:


def select_best(scores, seqs):
    selected_scores = []
    selected_seqs = []
    for i in range(len(scores[0])):
        best = scores[0][i]
        best_seq = seqs[0][i]
        for j in range(len(scores)-1):
            if scores[j+1][i] > best:
                best = scores[j+1][i]
                best_seq = seqs[j+1][i]
        selected_scores.append(best)
        selected_seqs.append(best_seq)

    return selected_seqs, selected_scores

if __name__ == '__main__':
    
    if OPT == 'FMRL':
        Optimize_FrameSlice = True
    else:
        Optimize_FrameSlice = False



    if Optimize_FrameSlice:
        model = load_framepool(mrl_path)


    else:

        model = torch.load(path,map_location=torch.device(device))['state_dict']  
        model.train()   
   



    wgan = tf.keras.models.load_model(gpath)

    """
    Data:
    """

    tf.random.set_seed(33)
    np.random.seed(33)

    diffs = []
    init_exps = []
    opt_exps = []
    orig_vals = []

    DIM = 40
    MAX_LEN = 128
    LR = np.exp(-LR)

    tempnoise = tf.random.normal(shape=[BATCH_SIZE,DIM])
    selectednoise = tempnoise

    best = 10

    LOW_START = False


    if LOW_START:
    
        for i in range(10000):
            tempnoise = tf.random.normal(shape=[BATCH_SIZE,DIM])
            sequences = wgan(tempnoise)

            seqs_gen = recover_seq(sequences, rev_rna_vocab)
            seqs_str = seqs_gen

            shape_ = tf.shape(np.array([encode_seq_framepool(seq) for seq in recover_seq(sequences, rev_rna_vocab)]))

            seqs = tf.convert_to_tensor(np.array([encode_seq_framepool(seq) for seq in recover_seq(sequences, rev_rna_vocab)]),dtype=tf.float32)

            
            pred =  model(seqs)

            t = tf.reshape(pred,(-1))
            t = t.numpy().astype('float')
            score = np.mean(t)

            if score < best:
                best = score
                selectednoise = tempnoise
        noise = tf.Variable(selectednoise)
    else:
        noise = tf.Variable(tf.random.normal(shape=[BATCH_SIZE,DIM]))
    

    noise_small = tf.random.normal(shape=[BATCH_SIZE,DIM],stddev=1e-4)

    optimizer = tf.keras.optimizers.Adam(learning_rate=np.power(np.e,LR))

    '''
    Optimization takes place here.
    '''

    bind_scores_list = []
    bind_scores_means = []
    sequences_list = []

    means = []
    maxes = []
    iters_ = []

    OPTIMIZE = True

    DNA_SEL = False


    sequences_init = wgan(noise)

    gen_seqs_init = sequences_init.numpy().astype('float')

    seqs_gen_init = recover_seq(gen_seqs_init, rev_rna_vocab)

    init_pos, init_neg = motif_count(seqs_gen_init)
    
    if Optimize_FrameSlice:
        seqs = prepare_framepool(seqs_gen_init)

        seqs_init = prepare_mttrans(seqs_gen_init)

        pred_init = model(seqs)
        
    else:


        one_hots = one_hot_all_motif(np.array(seqs_gen_init))
        seqs = torch.tensor(one_hots,dtype=torch.double)
        seqs = torch.transpose(seqs, 1, 2)
        seqs = seqs.float().to(device)


        pred_init = model.forward(seqs)
    
    if Optimize_FrameSlice:

        t = tf.reshape(pred_init,(-1))

        init_t = t.numpy().astype('float')
        
    else:
        
        t = torch.flatten(pred_init)
        t.float()
        
        init_t = t.cpu().detach().numpy()

    init_exp = np.mean(init_t)

    max_init = np.max(init_t)

    min_init = np.min(init_t)
    
    predicted_mrls = []

    STEPS = STEPS

    seqs_collection = []
    scores_collection = []
    if OPTIMIZE:
        iter_ = 0
        for opt_iter in tqdm(range(int(STEPS))):
            
            with tf.GradientTape() as gtape:
                gtape.watch(noise)
                sequences = wgan(noise)

                seqs_gen = recover_seq(sequences, rev_rna_vocab)
                seqs_collection.append(seqs_gen)
                seqs_str = seqs_gen
                
                if Optimize_FrameSlice:

                    seqs = tf.convert_to_tensor(np.array([encode_seq_framepool(seq) for seq in recover_seq(sequences, rev_rna_vocab)]),dtype=tf.float32)
                
                else:
                    seqs = torch.tensor(np.array(one_hot_all_motif(seqs_gen),dtype=np.float32))    

                if Optimize_FrameSlice:

                    with tf.GradientTape() as ptape:
                        ptape.watch(seqs)

                        pred =  model(seqs)
                        score = tf.reduce_mean(pred)
                        t = tf.reshape(pred,(-1))
                        mx = t.numpy().astype('float')
                        scores_collection.append(mx)
                        mx = np.max(mx)
                        
                        sum_ = tf.reduce_sum(t).numpy().astype('float')
                        
                        maxes.append(mx)
                        predicted_mrls.append(sum_/BATCH_SIZE)
                        means.append(sum_/BATCH_SIZE)

                    g1 = ptape.gradient(score,seqs)

                    OPTIMIZE_FULL = False
                    if OPTIMIZE_FULL:
                        tmp_g = g1.numpy().astype('float')
                        tmp_seqs = seqs_gen
                        tmp_lst = np.zeros(shape=(BATCH_SIZE,MAX_LEN,5))
                        for i in range(len(tmp_seqs)):
                            
                            len_ = len(tmp_seqs[i])
                            edited_g = tmp_g[i][:len_,:]
                            edited_g = np.pad(edited_g,((0,MAX_LEN-len_),(0,1)),'constant')   
                            tmp_lst[i] = edited_g   
                        
                        g1 = tf.convert_to_tensor(tmp_lst,dtype=tf.float32)

                    else:
                        
                        g1 = tf.pad(g1,tf.constant([[0, 0], [0, 0], [0, 1]]),"CONSTANT")

                    g1 = tf.math.scalar_mul(-1.0,g1)

                
                else:
                    
                    seqs = torch.transpose(seqs, 1, 2)
                    seqs = seqs.float()
                    seqs = torch.tensor(seqs.to(device), requires_grad=True)
                    pred = model(seqs)
                    pred = torch.flatten(pred)
                    predicted_mrls.append(np.average(pred.cpu().detach().numpy()))
                    scores_collection.append(pred.cpu().detach().numpy())
                    score = torch.mean(pred)
                    t = torch.flatten(pred)
                    mx = t.cpu().detach().numpy()
                    mx = np.max(mx)
                    
                    sum_ = torch.mean(t).cpu().detach().numpy()
                    
                    maxes.append(mx)
                    means.append(sum_/BATCH_SIZE)
                    pred.backward(torch.ones_like(pred))
                    
                    g1 = seqs.grad
                    
                    g1 = g1.cpu().detach().numpy()
                    g1 = tf.convert_to_tensor(g1)
                    g1 = tf.transpose(g1, perm=[0,2,1])
                    g1 = tf.pad(g1,tf.constant([[0, 0], [0, 0], [0, 1]]),"CONSTANT")
                    g1 = tf.math.scalar_mul(-1.0,g1)
                
                
                g2 = gtape.gradient(sequences,noise,output_gradients=g1)

            a1 = g2 + noise_small
            change = [(a1,noise)]
            optimizer.apply_gradients(change)

            iters_.append(iter_)
            iter_ += 1

        best_seqs, best_scores = select_best(scores_collection, seqs_collection)

        sequences_opt = wgan(noise)
        
        gen_seqs_opt = sequences_opt.numpy().astype('float')

        seqs_gen_opt = recover_seq(gen_seqs_opt, rev_rna_vocab)

        opt_pos, opt_neg = motif_count(seqs_gen_opt)
        
        if Optimize_FrameSlice:
            
            seqs_opt = prepare_framepool(seqs_gen_opt)


        
        else: 

            one_hots = np.array(one_hot_all_motif(seqs_gen_opt))
            # print(np.shape(one_hots))
            seqs = torch.tensor(one_hots,dtype=torch.double)
            seqs = torch.transpose(seqs, 1, 2)
            seqs = seqs.float().to(device)

        pred_opt = model(seqs)
        
        if Optimize_FrameSlice:

            t = tf.reshape(pred_opt,(-1))
            
            opt_t = t.numpy().astype('float')
            
        else:
            
            t = torch.flatten(pred_opt)
        
        
            opt_t = t.cpu().detach().numpy()

        opt_exp = np.mean(opt_t)

        min_opt = np.min(opt_t)
        max_opt = np.max(opt_t)

        with open(f'./outputs/init_mrl_{OPT}.txt', 'w') as f:
            f.writelines([str(x)+'\n' for x in init_t])

        with open(f'./outputs/opt_mrl_{OPT}.txt', 'w') as f:
            f.writelines([str(x)+'\n' for x in best_scores])

        with open(f'./outputs/opt_seqs_{OPT}.txt', 'w') as f:
            f.writelines([str(x)+'\n' for x in best_seqs])

        with open(f'./outputs/init_seqs_{OPT}.txt', 'w') as f:
            f.writelines([str(x)+'\n' for x in seqs_gen_init])
    

        print(f"Average Initial Pred: {np.average(init_t)}")
        print(f"Max Initial Pred: {np.max(init_t)}")
        print(f"Average Opt. Pred: {np.average(best_scores)}")
        print(f"Max Opt. Pred: {np.max(best_scores)}")



  0%|          | 0/100 [00:00<?, ?it/s]/var/folders/by/ns833bsx3yg5f8j5w0z5lmsw0000gq/T/ipykernel_88318/3924019252.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seqs = torch.tensor(seqs.to(device), requires_grad=True)
100%|██████████| 100/100 [01:44<00:00,  1.05s/it]


Average Initial Pred: -0.4125024378299713
Max Initial Pred: 0.45564794540405273
Average Opt. Pred: 0.4506695866584778
Max Opt. Pred: 1.3088536262512207


In [13]:
import requests
import json
import time

class GeneInfoRetriever:
    def __init__(self):
        self.base_url = "https://rest.ensembl.org"
        self.headers = {"Content-Type": "application/json"}
        self.sleep_time = 0.5  # To respect Ensembl API rate limits

    def get_gene_id(self, gene_symbol, species="human"):
        """Retrieve the Ensembl gene ID for a gene symbol."""
        species_map = {"human": "homo_sapiens", "mouse": "mus_musculus"}
        species_name = species_map.get(species.lower(), species)
        
        endpoint = f"/lookup/symbol/{species_name}/{gene_symbol}"
        response = self._make_request(endpoint)
        
        if response:
            return response.get("id")
        return None

    def get_gene_sequence(self, gene_id):
        """Retrieve the gene sequence for a gene ID."""
        endpoint = f"/sequence/id/{gene_id}?type=genomic"
        response = self._make_request(endpoint)
        
        if response:
            return response.get("seq")
        return None
        
    def get_promoter_sequence(self, gene_id, upstream=7000, downstream=3500):
        """
        Retrieve the promoter region sequence around the TSS.
        Default: 7kb upstream to 3.5kb downstream of TSS (Xpresso default)
        
        Parameters:
        gene_id (str): Ensembl gene ID
        upstream (int): Number of bases upstream of TSS
        downstream (int): Number of bases downstream of TSS
        
        Returns:
        str: The promoter sequence
        """
        # Get gene coordinates
        coords = self.get_gene_coordinates(gene_id)
        if not coords:
            return None
            
        chromosome = coords["chromosome"]
        strand = coords["strand"]
        
        # Calculate TSS position based on strand
        tss_position = coords["start"] if strand == 1 else coords["end"]
        
        # Calculate region to fetch based on strand
        if strand == 1:  # Forward strand
            seq_start = tss_position - upstream
            seq_end = tss_position + downstream
        else:  # Reverse strand
            seq_start = tss_position - downstream
            seq_end = tss_position + upstream
        
        # Ensure coordinates are positive
        seq_start = max(1, seq_start)
        
        # Get the sequence
        strand_str = "1" if strand == 1 else "-1"
        endpoint = f"/sequence/region/human/{chromosome}:{seq_start}..{seq_end}:{strand_str}"
        response = self._make_request(endpoint)
        
        if response:
            return response.get("seq")
        return None

    def get_gene_coordinates(self, gene_id):
        """Retrieve the genomic coordinates for a gene ID."""
        endpoint = f"/lookup/id/{gene_id}?expand=1"
        response = self._make_request(endpoint)
        
        if response:
            return {
                "chromosome": response.get("seq_region_name"),
                "start": response.get("start"),
                "end": response.get("end"),
                "strand": response.get("strand")
            }
        return None

    def get_tss_location(self, gene_id):
        """
        Retrieve the transcription start site (TSS) location.
        The TSS is typically at the start of the gene on the positive strand,
        or at the end of the gene on the negative strand.
        """
        coords = self.get_gene_coordinates(gene_id)
        
        if coords:
            # For genes on the positive strand, TSS is at the start
            # For genes on the negative strand, TSS is at the end
            tss = coords["start"] if coords["strand"] == 1 else coords["end"]
            return {
                "chromosome": coords["chromosome"],
                "position": tss,
                "strand": coords["strand"]
            }
        return None

    def get_transcript_info(self, gene_id):
        """Get information about all transcripts for a gene, including UTRs."""
        # This endpoint needs to be expanded to include UTRs
        endpoint = f"/lookup/id/{gene_id}?expand=1&utr=1"
        response = self._make_request(endpoint)
        
        transcripts = []
        if response and "Transcript" in response:
            for transcript in response["Transcript"]:
                # Extract basic transcript information
                transcript_info = {
                    "transcript_id": transcript.get("id"),
                    "start": transcript.get("start"),
                    "end": transcript.get("end"),
                    "strand": transcript.get("strand"),
                    "is_canonical": transcript.get("is_canonical", 0),
                    "biotype": transcript.get("biotype", ""),
                    "UTRs": []
                }
                
                # Extract UTR information based on actual Ensembl API response structure
                if "UTR" in transcript:
                    for utr in transcript["UTR"]:
                        utr_type = "5' UTR" if utr.get("object_type") == "five_prime_UTR" else "3' UTR"
                        transcript_info["UTRs"].append({
                            "start": utr.get("start"),
                            "end": utr.get("end"),
                            "type": utr_type
                        })
                
                # Add exon information if present
                if "Exon" in transcript:
                    transcript_info["Exon"] = transcript["Exon"]
                
                transcripts.append(transcript_info)
        
        # Sort transcripts so canonical is first if present
        transcripts.sort(key=lambda x: x["is_canonical"], reverse=True)
                
        return transcripts
        
    def get_canonical_transcript(self, gene_id):
        """Get the canonical transcript for a gene."""
        transcripts = self.get_transcript_info(gene_id)
        
        # First try to find the transcript marked as canonical
        for transcript in transcripts:
            if transcript.get("is_canonical") == 1:
                return transcript
                
        # If no canonical transcript is marked, look for protein_coding transcripts
        protein_coding = [t for t in transcripts if t.get("biotype") == "protein_coding"]
        if protein_coding:
            return protein_coding[0]
            
        # If no protein coding transcripts, return the first one
        return transcripts[0] if transcripts else None

    def _make_request(self, endpoint):
        """Make a request to the Ensembl REST API."""
        url = self.base_url + endpoint
        
        try:
            response = requests.get(url, headers=self.headers)
            time.sleep(self.sleep_time)  # Be nice to the API
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Error: {response.status_code} - {response.text}")
                return None
        except Exception as e:
            print(f"Request error: {e}")
            return None

    def calculate_sequence_features(self, sequence):
        """
        Calculate sequence-derived features for Xpresso input.
        
        Parameters:
        sequence (str): DNA sequence
        
        Returns:
        dict: Dictionary of sequence features
        """
        if not sequence:
            return None
            
        # Calculate GC content
        gc_count = sequence.upper().count('G') + sequence.upper().count('C')
        gc_content = gc_count / len(sequence)
        
        return {
            "gc_content": gc_content,
            "sequence_length": len(sequence)
        }
    
    def calculate_transcript_features(self, canonical_transcript):
        """
        Calculate transcript-derived features for Xpresso input.
        
        Parameters:
        canonical_transcript (dict): Canonical transcript information
        
        Returns:
        dict: Dictionary of transcript features
        """
        if not canonical_transcript:
            return {
                "five_prime_utr_length": 0,
                "three_prime_utr_length": 0,
                "total_utr_length": 0,
                "five_prime_utrs": [],
                "three_prime_utrs": [],
                "transcript_id": None
            }
            
        # Calculate UTR lengths
        five_prime_utrs = [utr for utr in canonical_transcript["UTRs"] if utr["type"] == "5' UTR"]
        three_prime_utrs = [utr for utr in canonical_transcript["UTRs"] if utr["type"] == "3' UTR"]
        
        five_prime_utr_length = sum(utr["end"] - utr["start"] + 1 for utr in five_prime_utrs) if five_prime_utrs else 0
        three_prime_utr_length = sum(utr["end"] - utr["start"] + 1 for utr in three_prime_utrs) if three_prime_utrs else 0
        
        # Calculate exon junction density
        exon_count = len(canonical_transcript.get("Exon", []))
        transcript_length = canonical_transcript["end"] - canonical_transcript["start"] + 1
        exon_junction_density = (exon_count - 1) / transcript_length if exon_count > 1 and transcript_length > 0 else 0
        
        return {
            "five_prime_utr_length": five_prime_utr_length,
            "three_prime_utr_length": three_prime_utr_length,
            "total_utr_length": five_prime_utr_length + three_prime_utr_length,
            "five_prime_utrs": five_prime_utrs,
            "three_prime_utrs": three_prime_utrs,
            "transcript_id": canonical_transcript.get("transcript_id"),
            "exon_count": exon_count,
            "transcript_length": transcript_length,
            "exon_junction_density": exon_junction_density
        }
    
    def get_all_gene_info(self, gene_symbol, species="human"):
        """Retrieve all information about a gene, including Xpresso features."""
        # Get the Ensembl gene ID
        gene_id = self.get_gene_id(gene_symbol, species)
        if not gene_id:
            return {"error": f"Gene {gene_symbol} not found"}
        
        # Get the gene sequence
        sequence = self.get_gene_sequence(gene_id)
        
        # Get the gene coordinates
        coordinates = self.get_gene_coordinates(gene_id)
        
        # Get the TSS location
        tss = self.get_tss_location(gene_id)
        
        # Get transcript information (including UTRs)
        transcripts = self.get_transcript_info(gene_id)
        
        # Get the canonical transcript
        canonical_transcript = self.get_canonical_transcript(gene_id)
        
        # Get the promoter sequence (7kb upstream to 3.5kb downstream of TSS)
        promoter_sequence = self.get_promoter_sequence(gene_id)
        
        # Calculate sequence features
        sequence_features = self.calculate_sequence_features(promoter_sequence)
        
        # Calculate transcript features from the canonical transcript
        transcript_features = self.calculate_transcript_features(canonical_transcript)
        
        return {
            "gene_symbol": gene_symbol,
            "gene_id": gene_id,
            "sequence": sequence,
            "promoter_sequence": promoter_sequence,
            "coordinates": coordinates,
            "tss": tss,
            "transcripts": transcripts,
            "canonical_transcript": canonical_transcript,
            "sequence_features": sequence_features,
            "transcript_features": transcript_features
        }

def one_hot_encode_sequence(sequence):
    """
    One-hot encode a DNA sequence.
    
    Parameters:
    sequence (str): DNA sequence
    
    Returns:
    list: One-hot encoded sequence (list of lists)
    """
    # Define the mapping
    mapping = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'N': [0, 0, 0, 0]  # For unknown bases
    }
    
    # Convert sequence to uppercase and one-hot encode
    sequence = sequence.upper()
    encoded = []
    for base in sequence:
        # Use N encoding for any non-standard bases
        if base not in mapping:
            base = 'N'
        encoded.append(mapping[base])
    
    return encoded

def export_for_xpresso(gene_info, output_file="xpresso_input.npz"):
    """
    Export gene information in a format suitable for Xpresso.
    
    Parameters:
    gene_info (dict): Gene information dictionary
    output_file (str): Output filename
    
    Returns:
    bool: Success status
    """
    try:
        # Extract the promoter sequence
        promoter_seq = gene_info["promoter_sequence"]
        if not promoter_seq:
            print("Error: No promoter sequence available")
            return False
        
        # One-hot encode the sequence
        one_hot_seq = one_hot_encode_sequence(promoter_seq)
        
        # Extract additional features from canonical transcript
        five_prime_utr_length = gene_info["transcript_features"]["five_prime_utr_length"]
        three_prime_utr_length = gene_info["transcript_features"]["three_prime_utr_length"]
        gc_content = gene_info["sequence_features"]["gc_content"]
        exon_junction_density = gene_info["transcript_features"]["exon_junction_density"]
        
        # Create a dictionary of features
        features = {
            "one_hot_sequence": one_hot_seq,
            "five_prime_utr_length": five_prime_utr_length,
            "three_prime_utr_length": three_prime_utr_length,
            "gc_content": gc_content,
            "exon_junction_density": exon_junction_density,
            "gene_id": gene_info["gene_id"],
            "gene_symbol": gene_info["gene_symbol"],
            "transcript_id": gene_info["transcript_features"]["transcript_id"],
            "tss_position": gene_info["tss"]["position"],
            "chromosome": gene_info["tss"]["chromosome"]
        }
        
        # Save to file (using a simple JSON format for now)
        import json
        with open(output_file.replace(".npz", ".json"), "w") as f:
            # Convert numpy arrays to lists for JSON serialization
            json_features = features.copy()
            json_features["one_hot_sequence"] = one_hot_seq
            json.dump(json_features, f, indent=2)
        
        print(f"Exported Xpresso input data to {output_file.replace('.npz', '.json')}")
        
        # Note: For a real .npz file (numpy array format), you would use:
        # import numpy as np
        # np.savez(output_file, 
        #    one_hot_sequence=np.array(one_hot_seq),
        #    five_prime_utr_length=five_prime_utr_length,
        #    three_prime_utr_length=three_prime_utr_length,
        #    gc_content=gc_content,
        #    exon_junction_density=exon_junction_density)
        
        return True
        
    except Exception as e:
        print(f"Error exporting Xpresso input: {e}")
        return False

def main():
    # Predefined variables - change these values as needed
    gene_symbol = "BRCA1"  # Example gene
    species = "human"      # Default species
    
    retriever = GeneInfoRetriever()
    result = retriever.get_all_gene_info(gene_symbol, species)
    
    # Print results
    if "error" in result:
        print(result["error"])
    else:
        print(f"Gene: {result['gene_symbol']} (Ensembl ID: {result['gene_id']})")
        print("\nGenome Coordinates:")
        print(f"Chromosome: {result['coordinates']['chromosome']}")
        print(f"Start: {result['coordinates']['start']}")
        print(f"End: {result['coordinates']['end']}")
        print(f"Strand: {'+' if result['coordinates']['strand'] == 1 else '-'}")
        
        print("\nTranscription Start Site (TSS):")
        print(f"Chromosome: {result['tss']['chromosome']}")
        print(f"Position: {result['tss']['position']}")
        
        print("\nXpresso Input Features:")
        print(f"Promoter sequence length: {len(result['promoter_sequence'])} bp")
        print(f"GC content: {result['sequence_features']['gc_content']:.2f}")
        print(f"5' UTR length: {result['transcript_features']['five_prime_utr_length']} bp")
        print(f"3' UTR length: {result['transcript_features']['three_prime_utr_length']} bp")
        
        print("\nCanonical Transcript:")
        canonical = result['canonical_transcript']
        if canonical:
            print(f"Transcript ID: {canonical['transcript_id']}")
            print(f"Start: {canonical['start']}")
            print(f"End: {canonical['end']}")
            print(f"Is Canonical: {'Yes' if canonical.get('is_canonical') == 1 else 'No'}")
            
            if canonical['UTRs']:
                print("\nCanonical UTRs:")
                for utr in canonical['UTRs']:
                    print(f"  {utr['type']}: {utr['start']}-{utr['end']} (length: {utr['end'] - utr['start'] + 1} bp)")
            
            # Summary of UTR lengths
            five_prime_length = result['transcript_features']['five_prime_utr_length']
            three_prime_length = result['transcript_features']['three_prime_utr_length']
            print(f"\nTotal 5' UTR length: {five_prime_length} bp")
            print(f"Total 3' UTR length: {three_prime_length} bp")
            print(f"Total UTR length: {five_prime_length + three_prime_length} bp")
            
            # Show exon information
            exon_count = result['transcript_features']['exon_count']
            if exon_count > 0:
                print(f"\nExon count: {exon_count}")
                print(f"Exon junction density: {result['transcript_features']['exon_junction_density']:.6f}")
        
        print("\nAll Transcripts:")
        for i, transcript in enumerate(result['transcripts'], 1):
            print(f"\nTranscript {i}: {transcript['transcript_id']} {'(CANONICAL)' if transcript.get('is_canonical') == 1 else ''}")
            print(f"Start: {transcript['start']}")
            print(f"End: {transcript['end']}")
            
            if transcript['UTRs']:
                print("UTRs:")
                for utr in transcript['UTRs']:
                    print(f"  {utr['type']}: {utr['start']}-{utr['end']}")
        
        print("\nPromoter Sequence (first 100 bp):")
        if result['promoter_sequence']:
            print(result['promoter_sequence'][:100] + "...")
            
        # Export data for Xpresso
        export_for_xpresso(result)

if __name__ == "__main__":
    main()

Gene: BRCA1 (Ensembl ID: ENSG00000012048)

Genome Coordinates:
Chromosome: 17
Start: 43044295
End: 43170245
Strand: -

Transcription Start Site (TSS):
Chromosome: 17
Position: 43170245

Xpresso Input Features:
Promoter sequence length: 10501 bp
GC content: 0.46
5' UTR length: 113 bp
3' UTR length: 1383 bp

Canonical Transcript:
Transcript ID: ENST00000357654
Start: 43044295
End: 43125364
Is Canonical: Yes

Canonical UTRs:
  5' UTR: 43125271-43125364 (length: 94 bp)
  5' UTR: 43124097-43124115 (length: 19 bp)
  3' UTR: 43044295-43045677 (length: 1383 bp)

Total 5' UTR length: 113 bp
Total 3' UTR length: 1383 bp
Total UTR length: 1496 bp

Exon count: 23
Exon junction density: 0.000271

All Transcripts:

Transcript 1: ENST00000357654 (CANONICAL)
Start: 43044295
End: 43125364
UTRs:
  5' UTR: 43125271-43125364
  5' UTR: 43124097-43124115
  3' UTR: 43044295-43045677

Transcript 2: ENST00000497488 
Start: 43044295
End: 43125300
UTRs:
  5' UTR: 43125271-43125300
  5' UTR: 43094643-43094860
  3